#### Análise de perfil de produtos com Python

##### Bibliotecas

In [19]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from anytree import Node, RenderTree

##### Relacionamento entre as bases *fato* e *dimensões*

In [47]:
#Importando a tabela fato união entre Cabecalho e Detalhes

fatos = pd.read_csv("fatos.csv", decimal = ",")

In [48]:
#Importando as tabelas dimensões

#Dimensão Categoria
d_cat = pd.read_excel("Dimensoes_DadosModelagem.xlsx", sheet_name=3)
#Dimensão Produto
d_prod = pd.read_excel("Dimensoes_DadosModelagem.xlsx", sheet_name=4)
#Dimensão Funcionários
d_func = pd.read_excel("Dimensoes_DadosModelagem.xlsx", sheet_name= 1)
#Dimensão Escritórios
d_esc = pd.read_excel("Dimensoes_DadosModelagem.xlsx", sheet_name= 2)


In [49]:
#Relacionamento: entre d_prod e d_cat
data = pd.merge(left = d_prod, right = d_cat, on = ["CategoriaID"], how = "inner")

#Relacionamento: entre data e f_cab
data = pd.merge(left = data, right = fatos, on = ["ProdutoID"], how = "inner")

#Relacionamento: entre data e d_func
data = pd.merge(left = data, right = d_func,  on = ["FuncionarioID"], how = "inner")

#Relacionamento: entre data e d_esc
data = pd.merge(left = data, right = d_esc,  left_on = ["Escritorio"], right_on = ["Escritorios"], how = "inner")

##### Descrição do período analisado

Data inicial e final da análise:

In [50]:
#Convertendo a coluna Data em data
data['Data']= pd.to_datetime(data['Data'], format = "%d/%m/%Y")

print("Data inicial:", min(data['Data']))

print("Data final:", max(data['Data']))

Data inicial: 2014-07-30 00:00:00
Data final: 2018-04-07 00:00:00


Criando variáveis para ano e meses:

In [51]:
#Criando a variável ano
data['Ano'] = pd.DatetimeIndex(data['Data']).year

#Criando a variável mês
data['Mês'] = pd.DatetimeIndex(data['Data']).month

Descobrindo quais os meses de venda para cada ano:

In [52]:
data.groupby("Ano")['Mês'].apply(lambda x: list(np.unique(x)))

Ano
2014                      [7, 8, 9, 10, 11, 12]
2015    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2016    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2017    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2018                               [1, 2, 3, 4]
Name: Mês, dtype: object

É importante destacar aqui que os dados de 2014 e 2018 estão incompletos. Assim possivelmente seus resultados financeiros relativos a produtos serão menores. 

##### Hierarquia para identificar departamentos e produtos

In [358]:
#Selecioando as coluanas referentes a departamento e produto
df = data.copy()
df = df[['Departamento', 'Produto']]

#Transformando dataframe em lista
df = df.values.tolist()


In [359]:
#Definindo uma função que destaca a segmentação entre departamento e produtos

def add_nodes(nodes, parent, child):
    if parent not in nodes:
        nodes[parent] = Node(parent)  
    if child not in nodes:
        nodes[child] = Node(child)
    nodes[child].parent = nodes[parent]


df = pd.DataFrame(columns=["Parent","Child"], data= df)
nodes = {}  
for parent, child in zip(df["Parent"],df["Child"]):
    add_nodes(nodes, parent, child)

roots = list(df[~df["Parent"].isin(df["Child"])]["Parent"].unique())
for root in roots:  
    for pre, _, node in RenderTree(nodes[root]):
        print("%s%s" % (pre, node.name))


Clothing
├── Lenin Jeansshorts
├── Mr X Trousers
├── O-Man Underwear
├── Desperado Jeans
├── Samba Socker  Socks
├── US-Master Jeans
├── Atles Lussekofta
├── Bow tie
├── Chantell Shirt
├── Oyaki Kimono
├── Shagall Socks
├── Tuxedo
├── Sumi Underwear
├── Le Baby Dress
├── Terence Top
├── Okkaba Skin Jackets
├── Jumpin Jack Flash Dress
├── Serve-Shirt
├── Stretch oui-pants
├── Skirt
├── Conserve Shoes
├── RDL Suit
├── Nikee Running Shoes
├── Sapporoo Gloves
├── Aino Shoes
├── Mehmet-Tröja
├── Duck Trousers
├── Duck Shirt
├── Rodbye Troje
├── Mehmet-Napp
├── Rossi Bermuda Shorts
├── Rossi Bikkini
├── Baywatch Bikkini
├── LA. Shorts
├── Summer Shorts
├── Rossi Shorts
├── Wimbledon T-Shirt
├── Squash Shorts
├── Tennis Suit
├── Basket Shoes
├── Basket Vest
├── Bike Helmet
├── Car Boots
├── Finnish Sport Blades
├── Finnish Swimsuit
├── Patamonia Fleece Jacket
├── Feiss Fleece Trousers
├── Rasta WCT
├── Game Over T-Shirt
├── Adihash Running Shoe
├── Sheat Shoes
├── Cap
├── Racing Truck  Socks


#### Produto e quantidade vendida

##### Quantidade de produtos por departamento

In [365]:
produtos = data.groupby(['Departamento']).Produto.agg(Quantidade = 'nunique')
produtos.loc['Total'] = produtos.sum()
produtos

,Quantidade
Departamento,
Clothing,63
Shoes,14
Total,77


O portifólio da companhia analisada contêm 77 produtos, sendo 63 correspondentes ao departamento **Clothing** e 14 ao departamento **Shoes**.

##### Quantidade de produtos vendidos por ano e departmento

In [364]:
produtos = data.groupby(['Ano','Departamento']).Produto.agg(Quantidade = 'count')
produtos.loc['Total'] = produtos.sum()
produtos

,Quantidade
"(2014, Clothing)",32
"(2014, Shoes)",4
"(2015, Clothing)",294
"(2015, Shoes)",52
"(2016, Clothing)",503
"(2016, Shoes)",84
"(2017, Clothing)",852
"(2017, Shoes)",189
"(2018, Clothing)",135
"(2018, Shoes)",27


Diante aos resultados observa-se que o departamento **Clothing** além de ter o maior número de produtos dentro da empresa também é responsável pela maior quantidade vendida de produtos em todos os anos. Destaque-se o ano de 2017 que obteve o melhor resultado quantidade vendidas para os dois departamentos.  

##### Os 5 produtos mais vendidos por ano e suas respectivas quantidades

In [366]:
data.groupby(['Ano', 'Produto']).Produto.agg(Qtde = 'count').sort_values(['Ano','Qtde'],  ascending=False).groupby(level=0).head(5)

Qtde
Ano  Produto                    
2018 Feiss Fleece Trousers     8
     Rasta WCT                 6
     Balett Shoes              5
     Bow tie                   5
     Minnki Pälsii             5
2017 Samba Socker  Socks      31
     Game Over T-Shirt        29
     Duck Shirt               27
     Aino Shoes               25
     High Heels Shoes         25
2016 Rossi Bermuda Shorts     21
     Bow tie                  17
     Rasta WCT                16
     Sheat Shoes              16
     Wimbledon T-Shirt        16
2015 Car Boots                11
     Desperado Jeans          10
     Rossi Bermuda Shorts     10
     Bike Helmet               9
     Duck Trousers             9
2014 Bike Helmet               2
     Mehmet-Tröja              2
     Okkaba Skin Jackets       2
     Rasta WCT                 2
     Rossi Shorts              2

##### Os 5 produtos mais vendidos no departamento **Clothing** durante todo o periodo analisado

In [367]:
data.query('Departamento == "Clothing"').groupby('Produto').Produto.agg(Qtde = 'count').sort_values('Qtde',  ascending=False).head()

,Qtde
Produto,
Rasta WCT,54
Samba Socker Socks,52
Rossi Bermuda Shorts,51
Game Over T-Shirt,51
Sheat Shoes,49


##### Os 5 produtos que mais vendem no departamento **Shoes** durante todo o periodo analisado

In [368]:
data.query('Departamento == "Shoes"').groupby('Produto').Produto.agg(Qtde = 'count').sort_values('Qtde', ascending=False).head()

,Qtde
Produto,
High Heels Shoes,50
Snake Boots,40
TieBreak Tennis shoes,37
Davenport,32
Balett Shoes,31


##### O produto que mais vendeu em cada ano no departamento **Shoes**

In [273]:
data.query('Departamento == "Shoes"').groupby(['Ano', 'Produto']).Produto.agg(Qtde = 'count').sort_values(['Ano','Qtde'], ascending=False).groupby(level=0).head(1)

,,Qtde
Ano,Produto,
2018,Balett Shoes,5
2017,High Heels Shoes,25
2016,High Heels Shoes,15
2015,High Heels Shoes,8
2014,DSW,1


No departamento **Shoes** o produto destaque corresponde ao **High Heels Shoes** cujo qual foi o recorde de venda de 2017 a 2015. 

##### O produto que mais vendeu em cada ano no departamento **Clothing**

In [282]:
data.query('Departamento == "Clothing"').groupby(['Ano', 'Produto']).Produto.agg(Qtde = 'count').sort_values(['Ano','Qtde'], ascending=False).groupby(level=0).head(1)

,,Qtde
Ano,Produto,
2018,Feiss Fleece Trousers,8
2017,Samba Socker Socks,31
2016,Rossi Bermuda Shorts,21
2015,Car Boots,11
2014,Bike Helmet,2


Não foi possível um produto destaque para o departmento **Clothing** tendo em vista que em todos os anos um produto diferente prevaleceu em 1º lugar de vendas.

##### Quantidade de produtos vendidos por escritório em cada ano 

In [369]:
data.groupby(['Ano', 'EscritorioLocalizacao']).Produto.agg(Qtde = 'count').sort_values(['Ano','Qtde'],  ascending=False).groupby(level=0).head(5)

Qtde
Ano  EscritorioLocalizacao      
2018 Stockholm                74
     Lund                     32
     Paris                    31
     Seattle                  18
     Nice                      7
2017 Stockholm               401
     Lund                    276
     Seattle                 177
     Paris                   128
     Nice                     59
2016 Stockholm               216
     Lund                    147
     Seattle                 104
     Paris                    74
     Nice                     46
2015 Stockholm               167
     Seattle                  74
     Lund                     55
     Nice                     28
     Paris                    22
2014 Stockholm                25
     Lund                      9
     Nice                      2

A partir desses dados podemos visualizar a expansão de dois escritórios em lugares diferentes (Seattle e Paris) de 2014 para 2015. Além disso, visualiza-se que o escritório em Stockholm é o responsável pelo maior número de vendas independentemente do ano analisado.  

##### O produto mais vendido por escritório e a sua respectividade quantidade 

In [289]:
data.groupby(['Ano', 'EscritorioLocalizacao', 'Produto']).Produto.agg(Qtde = 'count').sort_values(['Ano','EscritorioLocalizacao'], ascending=False).groupby(level=[0,1]).head(1)

Qtde
Ano  EscritorioLocalizacao Produto                    
2018 Stockholm             Aino Shoes                3
     Seattle               Aino Shoes                1
     Paris                 Baby Dark Lounge Suit     1
     Nice                  Bow tie                   1
     Lund                  Atles Lussekofta          1
2017 Stockholm             Adihash Running Shoe      5
     Seattle               Adihash Running Shoe      2
     Paris                 Adihash Running Shoe      3
     Nice                  Aino Shoes                1
     Lund                  Adihash Running Shoe      6
2016 Stockholm             Adihash Running Shoe      2
     Seattle               Aino Shoes                1
     Paris                 Adihash Running Shoe      1
     Nice                  Aino Shoes                1
     Lund                  Adihash Running Shoe      2
2015 Stockholm             Adihash Running Shoe      2
     Seattle               Adihash Running Shoe      1
     Paris                 Conserve Shoes            1
     Nice                  Balett Shoes              1
     Lund                  Aino Shoes                3
2014 Stockholm             Aino Shoes                1
     Nice                  Cap                       1
     Lund                  Bike Helmet               1

#### Produtos e os resultados financeiros

##### O produto que gerou o maior faturamento em cada ano, o respectivo valor e a proporção em relação ao faturamento total

In [7]:
#Criando a variável Faturamento
data['Faturamento'] = data["Valor"] * data["Quantidade"]

In [43]:
df_maiores = data.groupby(['Ano','Produto']).Faturamento.agg(Faturamento = 'sum')
df_maiores ['%'] = 100 * df_maiores['Faturamento'] / df_maiores.groupby('Produto')['Faturamento'].transform('sum')
df_maiores.sort_values(['Ano', 'Faturamento'],ascending=False).groupby('Ano').head(1)

,,Faturamento,%
Ano,Produto,,
2018,Minnki Pälsii,2138635.54,12.430174
2017,Minnki Pälsii,8355809.50,48.565622
2016,Minnki Pälsii,6622814.48,38.493111
2015,Okkaba Skin Jackets,1446044.54,66.661924
2014,Okkaba Skin Jackets,73065.68,3.368291


Observamos aqui que apesar do produto **Minnki Pälsii** não aparecer sempre como o produto com a maior quantidade vendida, tem-se o resultado que o mesmo foi o responsável pelo maior faturamento entre 2016 a 2017. Outro destaque é em relação ao produto **Okkaba Skin Jackets** que especialmente 2015 representou 66,66% do faturamento total. 

##### Conhecendo a margem bruta por produto em cada ano e o seu respectivo valor 

In [41]:
#Variável lucro bruto
data["LucroBruto"] = data["Valor"] - data["Custo"]

#Variável margem de lucro bruto
data['Margem Bruta'] = (data["LucroBruto"]/data["Valor"])*100

##### Descrição da margem bruta dos produtos

In [42]:
data['Margem Bruta'].describe()

count    2172.000000
mean       19.187783
std         6.809344
min         2.424682
25%        13.600000
50%        19.000000
75%        23.304348
max        35.211621
Name: Margem Bruta, dtype: float64

##### Produtos com maior margem bruta por ano 

In [18]:
df = data[["Ano","Produto", "Margem Bruta"]]
df.sort_values(['Margem Bruta', 'Ano'],ascending=False).groupby('Ano').head(1)

,Ano,Produto,Margem Bruta
1017,2017,Rodbye Troje,35.211621
421,2016,Squash Shorts,35.204397
678,2018,Snake Boots,35.200080
794,2015,Basket Vest,33.762641
122,2014,Rasta WCT,30.881007


Aqui visualizarmos que os produtos com maiores margem bruta de lucro não são aqueles que influenciaram no maior faturamento da empresa. 